## Two Body Loss

In [1]:
using KeldyshContraction
using KeldyshContraction: Plus, Minus

## System and regularization

The interaction action of elastic two body scattering, is defined as
$$
S_\mathrm{int} = i\Gamma \int d^d x \, [\frac{1}{2}(\bar{\phi}_+\phi_+)^2 +\frac{1}{2} (\bar{\phi}_-\phi_-)^2 -\bar{\phi}_-^2\phi_+^2]
$$
Above interaction can typically represent s-wave scattering of bosons.

In the RAK basis, this gives
$$
S_\mathrm{int} = i\Gamma\int d^d x \, [\frac{1}{2}(\bar{\phi}_c\bar{\phi}_q(\phi_c^2+\phi_q^2)-\phi_c\phi_q(\bar{\phi}_c^2+\bar{\phi}_q^2)+2\bar{\phi}_c\bar{\phi}_q\phi_c\phi_q)]
$$

A good check if the interaction Lagrangian is a valid physical process, is to check if the
normalization identity $Z=1$ holds. We can do this perturbatively in $g$ by expanding
$\exp(i S_\mathrm{int})$  and showing the average of the linear part of the system is zero
$$
\langle S_\mathrm{int}\rangle =  \langle S_\mathrm{int}^2\rangle  =\ldots = 0
$$
In computing the average, one performs Wick contractions to describe the average in terms
of the two-point correlators of the linear part of the system. However, in this case we
don't find that the vaccuum expectation value of the interaction Lagrangian is zero:

In [2]:
@qfields c::Destroy(Classical) q::Destroy(Quantum)

loss2boson_unregular = im*(
    0.5 * c' * q' * (c^2 + q^2) - 0.5 * c * q * ((c')^2 + (q')^2) + c' * q' * (c * q + c * q)
)

wick_contraction(loss2boson_unregular; simplify=true)

(-0.0 - 2.0im)*Gᴷ(y₁,y₁)*Gᴿ(y₁,y₁)

To make the interaction Lagrangian physically meaningful, we must regularize it by properly
handling equal-space-time propagators. These equal-time arguments emerge from the continuum
limit of the field theory but are naturally absent in discrete-time formulations
[(Gerbino et al, 2024)](https://arxiv.org/abs/2406.20028).
In the discrete picture, operators act on coherent states at adjacent (but distinct) time
slices during path integral construction via Trotter decomposition. To ensure all disconnected
diagrams vanish identically, we introduce a finite time-shift regularization ε > 0 for the
quantum jump operators, motivated by the underlying discrete Trotter structure. One gets:
$$
\bar{L}_+(t)L_+(t) \to \bar{L}_+(t)L_+(t-\epsilon) \quad, \bar{L}_-(t)L_-(t) \to \bar{L}_-(t)L_-(t+\epsilon)
\qaud \text{and} \quad
\bar{L}_-(t)L_+(t) \to \frac{1}{2}(\bar{L}_-(t)L_+(t+\epsilon) + \bar{L}_-(t)L_+(t-\epsilon))
$$
Applying this regularization to the interaction Lagrangian, we get:

In [3]:
loss2boson = im*(
    0.5 * c' * q' * (c(Minus) * c(Minus) + q(Minus) * q(Minus)) -
    0.5 * c(Plus) * q(Plus) * (c' * c' + q' * q') +
    c' * q' * (c(Plus) * q(Plus) + c(Minus) * q(Minus))
)
L_int = InteractionLagrangian(loss2boson)

Interaction Lagrangian with fields c and q:
((0.0 + 0.5im)*(c⁻*c⁻*̄q*̄c)+(0.0 + 0.5im)*(q⁻*q⁻*̄q*̄c)+(-0.0 - 0.5im)*(q⁺*c⁺*̄c*̄c)+(-0.0 - 0.5im)*(q⁺*c⁺*̄q*̄q)+(0.0 + 1.0im)*(c⁺*q⁺*̄c*̄q)+(0.0 + 1.0im)*(c⁻*q⁻*̄c*̄q))

Indeed, the vacuum expectation value of the interaction Lagrangian is now zero:

## First order Green's function

In [4]:
GF = DressedPropagator(L_int)

Dressed Propagator:
keldysh:  -2.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,x₂) + Gᴿ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴷ(y₁,x₂) + -1.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴷ(y₁,x₂) + Gᴷ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,x₂) + 2.0*Gᴿ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴬ(y₁,x₂) + -1.0*Gᴷ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴬ(y₁,x₂)
retarded: -1.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴿ(y₁,x₂) + Gᴿ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴿ(y₁,x₂)
advanced: -1.0*Gᴬ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴬ(y₁,x₂) + Gᴬ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,x₂)

In [5]:
Σ = SelfEnergy(GF)

Self Energy:
keldysh:  -2.0*Gᴿ(y₁,y₁) + 2.0*Gᴷ(y₁,y₁)
retarded: -1.0*Gᴿ(y₁,y₁) + Gᴷ(y₁,y₁)
advanced: Gᴿ(y₁,y₁) + -1.0*Gᴷ(y₁,y₁)

The following indeed corresponds with what is reported in [(Gerbino et al, 2024)](https://arxiv.org/abs/2406.20028).

## Second order Green's function

In [6]:
GF = DressedPropagator(L_int; order = 2)

Dressed Propagator:
keldysh:  -2.0*Gᴷ(x₁,y₁)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂)*Gᴬ(y₁,x₂) + -0.5*Gᴿ(x₁,y₁)*Gᴷ(y₁,y₂)*Gᴷ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴬ(y₂,x₂) + 4.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂)*Gᴬ(y₁,x₂) + Gᴿ(x₁,y₁)*Gᴷ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₁)*Gᴬ(y₂,x₂) + -2.0*Gᴿ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴿ(y₁,y₂)*Gᴿ(y₂,y₂)*Gᴬ(y₂,x₂) + 2.0*Gᴿ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₂)*Gᴬ(y₂,x₂) + 2.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴷ(y₂,y₂)*Gᴬ(y₁,x₂) + 2.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴿ(y₁,y₂)*Gᴿ(y₂,y₂)*Gᴬ(y₂,x₂) + 2.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₂)*Gᴬ(y₂,x₂) + Gᴿ(x₁,y₁)*Gᴷ(y₁,y₁)*Gᴷ(y₁,y₂)*Gᴿ(y₂,y₂)*Gᴬ(y₂,x₂) + 2.0*Gᴷ(x₁,y₁)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂)*Gᴬ(y₁,x₂) + -1.0*Gᴷ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₂)*Gᴬ(y₂,x₂) + Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴷ(y₁,y₂)*Gᴷ(y₂,y₂)*Gᴬ(y₂,x₂) + Gᴷ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₂)*Gᴬ(y₂,x₂) + Gᴿ(x₁,y₁)*Gᴷ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴬ(y₂,x₂) + -2.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₁)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₂)*Gᴬ(y₂,x₂) + -1.0*Gᴿ(x₁,y₁)*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴿ(y₂,y₂)*Gᴷ(y₁,x₂) + Gᴷ(x₁,y₁)*Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ

In [7]:
Σ = SelfEnergy(GF; order = 2)

Self Energy:
keldysh:  Gᴷ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁) + Gᴷ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₁) + -0.5*Gᴿ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁) + -4.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂) + -2.0*Gᴷ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₁) + 2.0*Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂) + 2.0*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴷ(y₂,y₂) + 4.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + 2.0*Gᴷ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁) + -0.5*Gᴷ(y₁,y₂)*Gᴷ(y₁,y₂)*Gᴷ(y₂,y₁) + -2.0*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴿ(y₂,y₂) + -2.0*Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + -0.5*Gᴬ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₁)
retarded: -1.0*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴿ(y₂,y₂) + -1.0*Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + -0.5*Gᴷ(y₁,y₂)*Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁) + 2.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂) + -0.5*Gᴿ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁) + 2.0*Gᴷ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁) + Gᴷ(y₁,y₂)*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁) + -2.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂) + Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴷ(y₂,y₂)
advanced: Gᴷ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴿ(y₂,y₂) + Gᴷ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₁) + -2.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + -1.0*Gᴷ(

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*